In [0]:
%sql
use catalog `get_started`;
use schema `labuser`;

SHOW TABLES;

Simple Example of the Medallion Architecture
Data transtormation Overview
Objective: Create a pipeline that can be scheduled to run automatically. The pipeline will:
1. Ingest all CSV files from the myfiles volume and create a bronze table.
2. Prepare the bronze table by adding new columns and create a silver table.
3. Create a gold aggregated table for consumers.

### Bronze
#### Create a table using all csv file in myfiles volume


In [0]:
%sql
use catalog `get_started`;
use schema `labuser`;

-- drop table if exists current_employees
DROP TABLE IF EXISTS current_employees_bronze;

-- create an empty delta table and columns
CREATE TABLE IF NOT EXISTS current_employees_bronze (
    ID INT,
    Firstname STRING,
    Country STRING,
    Role STRING,
    InputFile STRING
) USING DELTA;
    
-- create the bronze raw ingestion table and include the csv file name for the rows
COPY INTO current_employees_bronze
FROM (
  SELECT *, _metadata.file_name AS InputFile
  FROM '/Volumes/get_started/labuser/myfiles/'
  )
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', "inferSchema" = 'true');


-- View bronze table
SELECT * FROM current_employees_bronze;

### Silver
Objective: Transform the bronze table and insert the resulting rows into the silver table.
1. Create and display a temporary view named temp_view_employees_silver from the current_employees_bronze table.
The view will:
- Select the columns ID, FirstName, Country.
- Convert the Role column to uppercase.
- Add two new columns: Timestamp and Date.

In [0]:
%sql
-- create a temp view
CREATE OR REPLACE TEMP VIEW temp_view_employees_silver AS
SELECT ID, Firstname, Country, upper(Role) as Role, current_timestamp() as TimeStamp, date(timestamp) as Date 
FROM current_employees_bronze;

-- View the results of the temp view
SELECT * FROM temp_view_employees_silver; 

In [0]:
%sql
-- drop table if exists current_employees
DROP TABLE IF EXISTS current_employees_silver;

-- create an empty delta table and columns
CREATE TABLE IF NOT EXISTS current_employees_silver (
    ID INT,
    Firstname STRING,
    Country STRING,
    Role STRING,
    TimeStamp TIMESTAMP,
    Date DATE
);
    
-- Insert records from the view when not matched with the target silver table
MERGE INTO current_employees_silver AS target
USING temp_view_employees_silver AS source
-- ON current_employees_silver.ID = temp_view_employees_silver.ID
ON target.ID = source.ID
WHEN NOT MATCHED THEN INSERT *;  -- insert records if not match


-- View bronze table
SELECT * FROM current_employees_silver;

### GOLD
Aggregate the silver table to create the final gold table

1. Create a temporary vieew name temp_view_total_roles that aggregates the total number of employees by role. Then display the results of the view

In [0]:
%sql
-- create a temp view
CREATE OR REPLACE TEMP VIEW temp_view_total_roles AS
SELECT Role,count(*) as TotalEmployees
FROM current_employees_silver
GROUP BY Role;

-- View the results of the temp view
SELECT * FROM temp_view_total_roles; 

In [0]:
%sql
-- drop table if exists current_employees
DROP TABLE IF EXISTS total_roles_gold;

-- create an empty delta table and columns
CREATE TABLE IF NOT EXISTS total_roles_gold (
    Role STRING,
    TotalEmployees INT
);

In [0]:
%sql
-- Insert overwrite to give it history rather than dropping it
INSERT OVERWRITE TABLE total_roles_gold
SELECT * FROM temp_view_total_roles;

In [0]:
%sql
SELECT * FROM total_roles_gold

### Data Governance and Security
Objectives: View the lineage of the total_roles gold table and learn how to set its permissions.

### Cleanup


In [0]:
%sql
-- Drop the temp view
DROP VIEW IF EXISTS temp_view_total_roles;
DROP VIEW IF EXISTS temp_view_employees_silver;

-- Drop tables
DROP TABLE IF EXISTS current_employees_bronze;
DROP TABLE IF EXISTS current_employees_silver;
DROP TABLE IF EXISTS total_roles_gold;